In [1]:
import os
import tensorflow as tf
import keras
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

In [2]:
# Specify the input image size for VGG19
img_size = (224, 224)

# Data Augmentation (commented out for comparison)
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     zoom_range=0.2,
#     horizontal_flip=True
# )

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Set the batch size
batch_size = 32

# Specify the paths to your train, validation, and test data
train_dir = 'D:\\Splitted Curated X-Ray Dataset\\train'
val_dir = 'D:\\Splitted Curated X-Ray Dataset\\val'
test_dir = 'D:\\Splitted Curated X-Ray Dataset\\test'

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Flow validation images in batches using val_datagen generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',

)

# Flow test images in batches using test_datagen generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=1,
    class_mode='categorical',
    shuffle = False
)

Found 7364 images belonging to 4 classes.
Found 920 images belonging to 4 classes.
Found 924 images belonging to 4 classes.


In [3]:
# Load the VGG-19 model with pretrained weights (excluding the top dense layers)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top
model = models.Sequential()

# Add the VGG-19 base model
model.add(base_model)

# Flatten the output layer to 1 dimension
model.add(layers.Flatten())

# Add a fully connected layer with 256 hidden units and ReLU activation
model.add(layers.Dense(256, activation='relu'))

# Add a dropout layer for regularization
model.add(layers.Dropout(0.5))

# Add the final output layer with 4 classes for classification
model.add(layers.Dense(4, activation='softmax'))

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 26,448,196
Trainable params: 6,423,812
Non-trainable params: 20,024,384
_________________________________________________________________


In [5]:
# Define the initial learning rate and the learning rate schedule
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

# Define the optimizer with the learning rate schedule
opt = optimizers.Adam(learning_rate=initial_learning_rate)

# Compile the model
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Define a custom learning rate scheduler function
def custom_lr_schedule(epoch):
    return initial_learning_rate * (0.96 ** (epoch / 10))

# Set up LearningRateScheduler callback with the custom function
lr_scheduler = LearningRateScheduler(custom_lr_schedule)

# Train the model with the specified number of epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=75,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[early_stop, lr_scheduler]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/75
230/230 [==============================] - 136s 585ms/step - loss: 0.5010 - accuracy: 0.7766 - val_loss: 0.4077 - val_accuracy: 0.8438 - lr: 0.0010
Epoch 2/75
230/230 [==============================] - 133s 579ms/step - loss: 0.5065 - accuracy: 0.7691 - val_loss: 0.4157 - val_accuracy: 0.8449 - lr: 9.9593e-04
Epoch 3/75
230/230 [==============================] - 132s 574ms/step - loss: 0.4850 - accuracy: 0.7872 - val_loss: 0.3866 - val_accuracy: 0.8538 - lr: 9.9187e-04
Epoch 4/75
230/230 [==============================] - 132s 573ms/step - loss: 0.4636 - accuracy: 0.7912 - val_loss: 0.3793 - val_accuracy: 0.8616 - lr: 9.8783e-04
Epoch 5/75
230/230 [==============================] - 132s 572ms/step - loss: 0.4544 - accuracy: 0.7923 - val_loss: 0.3787 - val_accuracy: 0.8616 - lr: 9.8380e-04
Epoch 6/75
230/230 [==============================] - 133s 577ms/step - loss: 0.4420 - accuracy: 0.7964 - val_loss: 0.3753 - val_accuracy: 0.8627 - lr: 9.7980e-04
Epoch 7/75
230/230 [======

In [6]:
from sklearn.metrics import classification_report

# Get predictions on the test set
y_pred = model.predict(test_generator)
y_true = test_generator.classes

# Convert predictions to class labels
y_pred_labels = tf.argmax(y_pred, axis=1)
class_labels = list(test_generator.class_indices.keys())

# Generate and print the classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_labels, target_names=class_labels, digits=4))

924/924 [==============================] - 19s 20ms/step

Classification Report:
                     precision    recall  f1-score   support

           COVID-19     0.9699    1.0000    0.9847       129
             Normal     0.9758    0.9878    0.9818       327
Pneumonia-Bacterial     0.7526    0.9502    0.8399       301
    Pneumonia-Viral     0.8375    0.4012    0.5425       167

           accuracy                         0.8712       924
          macro avg     0.8840    0.8348    0.8372       924
       weighted avg     0.8773    0.8712    0.8566       924



: 